In the following cells, we simplify the expression for the 27 functions in BETijk and we store them in `funclist`

In [ ]:
from sympy import simplify, cos, sin, exp, Symbol, lambdify, integrate, pi, sqrt
from sympy.matrices import *
from sympy.physics.quantum import TensorProduct

In [ ]:
x = Symbol('x')
y = Symbol('y')
z = Symbol('z')
c = Symbol('c', positive=True)

In [ ]:
Rz = Matrix([[cos(x),sin(x),0],[-sin(x),cos(x),0],[0,0,1]])
Ry = Matrix([[cos(z),0,-sin(z)],[0,1,0],[sin(z),0,cos(z)]])
Rc = Matrix([[cos(y),sin(y),0],[-sin(y),cos(y),0],[0,0,1]])

In [ ]:
RAI = Rz*Ry*Rc
RAI

In [ ]:
SAI = TensorProduct(RAI,RAI)
TAI = TensorProduct(SAI,RAI)
BETabc = Matrix([0, 0, 0.15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [ ]:
BETijk = TAI * BETabc

In [ ]:
funclist = []
for bet in BETijk:
    funclist.append(lambdify( (x,y,z), bet ))

In [ ]:
len(funclist)

$$
\frac{1}{N_E} = \int_0^\pi \exp{(-\theta/\theta_E)} \sin{(\theta)} \mathrm{d}\theta = \frac{\theta_E^2(\exp{(-\pi/\theta_E)}+1)}{\theta_E^2+1}
$$

In [ ]:
simplify(integrate(exp( - x / c )*sin(x), (x,0,pi)))

$$
\langle \theta \rangle = \frac{\theta_E^2+1}{\theta_E^2(\exp{(-\pi/\theta_E)}+1)} \int_0^\pi \theta \exp{(-\theta/\theta_E)} \sin{(\theta)} \mathrm{d}\theta = \frac{2 \theta_E}{\theta_E^2+1} + \frac{\pi}{\exp{(\pi/\theta_E)}+1}
$$

In [ ]:
simplify(integrate(exp( - x / c )*x*sin(x), (x,0,pi)) / integrate(exp( - x / c )*sin(x), (x,0,pi)))

The functions to carry out the numerical calculations are defined here below

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.optimize import brentq

In [ ]:
def funcKai(L):
    g = lambda z : functprim(z,L)*np.sin(z)
    x = np.linspace(0,2*np.pi,120)
    y = np.linspace(0,2*np.pi,120)
    z = np.linspace(0,np.pi,60)
    dx = x[1]-x[0]
    dy = y[1]-y[0]
    den = g(z).sum()
    x = x.reshape(-1,1,1)
    y = y.reshape(1,-1,1)
    z = z.reshape(1,1,-1)
    integrals = []
    for i in range(27):
        f = lambda x,y,z : funclist[i](x,y,z)*g(z)
        num = f(x,y,z).sum()*dx*dy
        if i==26: # the 27th is a function of two variables only
            num *= 120
        integrals.append(num/den)
    return np.array(integrals)

In [ ]:
def compute(L):
    Kai = funcKai(L)
    c = 299792458; v1 = 1/800*1*1e9; v2 = 368000
    na1=1; na2=1; na3=1
    nb1 = 1.329; nb2 = 1.16 + 0.02j; nb3 = 1.331
    nab1 = 1.15; nab2 = 1.09; nab3 = 1.15
    w1 = 2*np.pi*c*v1; w2 = 2*np.pi*c*v2; w3 = w1+w2
    psi1r = 63/180*np.pi; psi2r = 55/180*np.pi
    psi1t = np.arcsin(na1/nb1*np.sin(psi1r)); psi2t = np.arcsin(na2/nb2*np.sin(psi2r))
    psi3r = np.arcsin(1/(w3*na3)*(w1*na1*np.sin(psi1r)+w2*na2*np.sin(psi2r)))
    psi3t = np.arcsin(1/(w3*nb3)*(w1*nb1*np.sin(psi1t)+w2*nb2*np.sin(psi2t)))
    Lx1 = (2*na1*np.cos(psi1t))/(na1*np.cos(psi1t)+nb1*np.cos(psi1r))
    Lx2 = (2*na2*np.cos(psi2t))/(na2*np.cos(psi2t)+nb2*np.cos(psi2r))
    Lx3 = (2*na3*np.cos(psi3t))/(na3*np.cos(psi3t)+nb3*np.cos(psi3r))
    Ly1 = (2*na1*np.cos(psi1r))/(na1*np.cos(psi1r)+nb1*np.cos(psi1t))
    Ly2 = (2*na2*np.cos(psi2r))/(na2*np.cos(psi2r)+nb2*np.cos(psi2t)) 
    Ly3 = (2*na3*np.cos(psi3r))/(na3*np.cos(psi3r)+nb3*np.cos(psi3t)) 
    Lz1 = ((2*nb1*np.cos(psi1r))/(na1*np.cos(psi1t)+nb1*np.cos(psi1r)))*(na1/nab1)**2 
    Lz2 = ((2*nb2*np.cos(psi2r))/(na2*np.cos(psi2t)+nb2*np.cos(psi2r)))*(na2/nab2)**2 
    Lz3 = ((2*nb3*np.cos(psi3r))/(na3*np.cos(psi3t)+nb3*np.cos(psi3r)))*(na3/nab3)**2 
    Ep1 = np.array( [0, 1, 1, 0] )
    Ep2 = np.array( [1, 0, 1, 0] )
    Es1 = np.array( [1, 0, 0, 1] )
    Es2 = np.array( [0, 1, 0, 1] )
    EX1 = Lx1*np.cos(psi1r)*Ep1; 
    EX2 = Lx2*np.cos(psi2r)*Ep2;
    EY1 = Ly1*Es1; 
    EY2 = Ly2*Es2;
    EZ1 = Lz1*np.sin(psi1r)*Ep1; 
    EZ2 = Lz2*np.sin(psi2r)*Ep2;
    PX = Kai[0]*EX1*EX2+Kai[1]*EX1*EY2+Kai[2]*EX1*EZ2+Kai[3]*EY1*EX2+Kai[4]*EY1*EY2+Kai[5]*EY1*EZ2+Kai[6]*EZ1*EX2+Kai[7]*EZ1*EY2+Kai[8]*EZ1*EZ2;
    PY = Kai[9]*EX1*EX2+Kai[10]*EX1*EY2+Kai[11]*EX1*EZ2+Kai[12]*EY1*EX2+Kai[13]*EY1*EY2+Kai[14]*EY1*EZ2+Kai[15]*EZ1*EX2+Kai[16]*EZ1*EY2+Kai[17]*EZ1*EZ2;
    PZ = Kai[18]*EX1*EX2+Kai[19]*EX1*EY2+Kai[20]*EX1*EZ2+Kai[21]*EY1*EX2+Kai[22]*EY1*EY2+Kai[23]*EY1*EZ2+Kai[24]*EZ1*EX2+Kai[25]*EZ1*EY2+Kai[26]*EZ1*EZ2;
    ESFGs = Ly3*PY;
    ESFGp = - Lx3 * np.cos(psi3r) * PX + Lz3 * np.sin(psi3r) * PZ;
    ISFGs = ESFGs * ESFGs; ISFGp = ESFGp * ESFGp;
    return np.real(ISFGs[0]), np.real(ISFGs[1]), np.real(ISFGp[2])

### Exponential distribution function
The average angle, $\langle \theta \rangle$, is converted to $\theta_E$ using
$$
\langle \theta \rangle = N_E \int_0^\pi \theta \exp{(-\theta/\theta_E)} \sin{(\theta)} \mathrm{d}\theta = \frac{2 \theta_E}{\theta_E^2+1} + \frac{\pi}{\exp{(\pi/\theta_E)}+1}
$$
where $N_E$ is the normalization factor
$$
N_E = \frac{1}{ \int_0^\pi \exp{(-\theta/\theta_E)} \sin{(\theta)} \mathrm{d}\theta} = \frac{\theta_E^2+1}{\theta_E^2(\exp{(-\pi/\theta_E)}+1)}
$$

Here is a plot of $\theta_E$ as a function of $\langle \theta \rangle$ 

In [ ]:
decay2mean = lambda x : 2*x/(x**2+1) + np.pi/(np.exp(np.pi/x)+1)
x = np.arange(0.01,np.pi/2,0.01)
plt.plot(x/np.pi*180,decay2mean(x)/np.pi*180)
plt.xlabel('Average Angle  /  $\degree$')
plt.ylabel(r'$\theta_E$  /  $\degree$')
plt.xlim(0,90)
plt.savefig('decayVSaverage.pdf')

In [ ]:
def functprim(z,L):
    # L is the average angle
    # D is the decay constant
    f = lambda x : 2*x/(x**2+1) + np.pi/(np.exp(np.pi/x)+1) - L
    D = brentq(f, 0.01, 1e10) # finds the root of f in the interval 0.01 1e10
    return np.exp( - z / D )

In [ ]:
SSP = np.empty(0)
SPS = np.empty(0)
PPP = np.empty(0)
x = np.arange(0.02,np.pi/2,0.05)
for L in x:
    ssp, sps, ppp = compute(L)
    SSP = np.append(SSP,ssp)
    SPS = np.append(SPS,sps)
    PPP = np.append(PPP,ppp)

In [ ]:
plt.plot(x*180/np.pi,SSP,label='SSP',marker='o')
plt.plot(x*180/np.pi,SPS,label='SPS',marker='o')
plt.plot(x*180/np.pi,PPP,label='PPP',marker='o')
plt.legend(frameon=False)
plt.xlabel(r'$\langle \theta \rangle$  /  $\degree$')
plt.ylabel('SFG Intensity')
plt.ylim(0,10)
plt.xlim(0,90)
plt.title('Exponential Decay')
plt.savefig('expSFG.pdf')

We can also calculate the SFG intensity as a function of $\theta_E$

In [ ]:
def functprim(z,L):
    return np.exp( - z / L )

In [ ]:
SSP = np.empty(0)
SPS = np.empty(0)
PPP = np.empty(0)
x = np.linspace(0.01,np.pi*4,200)
for L in x:
    ssp, sps, ppp = compute(L)
    SSP = np.append(SSP,ssp)
    SPS = np.append(SPS,sps)
    PPP = np.append(PPP,ppp)

In [ ]:
plt.plot(x*180/np.pi,SSP,label='SSP',marker='o')
plt.plot(x*180/np.pi,SPS,label='SPS',marker='o')
plt.plot(x*180/np.pi,PPP,label='PPP',marker='o')
plt.legend(frameon=False)
plt.xlabel(r'$\theta_E$  /  $\degree$')
plt.ylabel('SFG Intensity')
plt.ylim(0,10)
plt.xlim(0,90)
plt.savefig('expSFG_thetaE.pdf')

The SFG intensity as a function of $\theta_E$ can be readily converted to SFG intensity vs $\langle \theta \rangle$

In [ ]:
plt.plot(decay2mean(x)*180/np.pi,SSP,label='SSP',marker='o')
plt.plot(decay2mean(x)*180/np.pi,SPS,label='SPS',marker='o')
plt.plot(decay2mean(x)*180/np.pi,PPP,label='PPP',marker='o')
plt.legend(frameon=False)
plt.xlabel('Average Angle  /  $\degree$')
plt.ylabel('SFG Intensity')
plt.ylim(0,10)
plt.xlim(0,90)

### Gaussian distribution function
We convert the average angle, $\langle \theta \rangle$, into $\theta_G$ using
$$
\langle \theta \rangle = \frac{N_G}{\sqrt{2 \pi \sigma^2}} \int_0^\pi \theta \exp{\frac{-(\theta-\theta_G)^2}{2 \sigma^2}} \sin{(\theta)} \mathrm{d}\theta 
$$
where $N_G$ is the normalization factor
$$
N_G = \frac{\sqrt{2 \pi \sigma^2}}{\int_0^\pi \exp{\frac{-(\theta-\theta_G)^2}{2 \sigma^2}} \sin{(\theta)} \mathrm{d}\theta}
$$
and $\sigma=15^\circ$.

In [ ]:
def functprim(z,L):
    sigma2 = (15/180*np.pi)**2
    return np.exp( -(z-L)**2 / (2*sigma2) )

In [ ]:
SSP = np.empty(0)
SPS = np.empty(0)
PPP = np.empty(0)
x = np.linspace(-2*np.pi,np.pi/2,100)
for L in x:
    ssp, sps, ppp = compute(L)
    SSP = np.append(SSP,ssp)
    SPS = np.append(SPS,sps)
    PPP = np.append(PPP,ppp)

In [ ]:
plt.plot(x*180/np.pi,SSP,label='SSP',marker='o')
plt.plot(x*180/np.pi,SPS,label='SPS',marker='o')
plt.plot(x*180/np.pi,PPP,label='PPP',marker='o')
plt.legend(frameon=False)
plt.xlabel(r'$\theta_G$  /  $\degree$')
plt.ylabel('SFG Intensity')
plt.ylim(0,10)
plt.xlim(0,90)
plt.savefig('gaussianSFG_thetaG.pdf')

Here $\theta_G$ is converted to $\langle \theta \rangle$

In [ ]:
def thetag2mean(L):
    x = np.linspace(0,np.pi,1000)
    sigma2 = (15/180*np.pi)**2
    num = lambda L : np.trapz(np.exp(-(x-L)**2/(2*sigma2))*x*np.sin(x),x)
    den = lambda L : np.trapz(np.exp(-(x-L)**2/(2*sigma2))*np.sin(x),x)
    return np.array(list(map(num, L)))/np.array(list(map(den, L)))

In [ ]:
plt.plot(thetag2mean(x)*180/np.pi,SSP,label='SSP',marker='o')
plt.plot(thetag2mean(x)*180/np.pi,SPS,label='SPS',marker='o')
plt.plot(thetag2mean(x)*180/np.pi,PPP,label='PPP',marker='o')
plt.legend(frameon=False)
plt.xlabel(r'$\langle \theta \rangle$  /  $\degree$')
plt.ylabel('SFG Intensity')
plt.ylim(0,10)
plt.xlim(0,90)
plt.title('Gaussian')
plt.savefig('gaussianSFG.pdf')